In [26]:
# import libraries
import pandas as pd
import numpy as np
from sqlalchemy import create_engine
import sqlite3
import re
import nltk
nltk.download(['punkt', 'wordnet'])
from nltk.stem.wordnet import WordNetLemmatizer
from nltk.tokenize import word_tokenize
from sklearn.pipeline import Pipeline
from sklearn.metrics import confusion_matrix, classification_report, precision_score, recall_score, f1_score
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
import pickle

[nltk_data] Downloading package punkt to /Users/jbryant/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /Users/jbryant/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [2]:
# load data from database
engine = create_engine('sqlite:///fictionTaggingEngine.db')
conn = sqlite3.connect('fictionTaggingEngine.db')

df = pd.read_sql('SELECT * FROM processedAO3data', conn)

lemmatizer = WordNetLemmatizer()

In [3]:
df

,title,author,story_text,url,rating,archiveWarnings,category,fandom,relationships,character,...,consolidatedFandom_theBellesTags,consolidatedFandom_thePowerTags,consolidatedFandom_trumanShowTags,consolidatedFandom_umbrellaAcademyTags,consolidatedFandom_underworldTags,consolidatedFandom_vagrantQueenTags,consolidatedFandom_wayfarersSeriesTags,consolidatedFandom_whiteCollarTags,consolidatedFandom_worstWitchTags,consolidatedFandom_wrinkleInTimeTags
0,One Too Many Accidents,kazesuke,"“Sophiiiieee!!!” Loud, wet footsteps slammed a...",http://archiveofourown.org/works/299981,General Audiences,No Archive Warnings Apply,Gen,Howl no Ugoku Shiro | Howl's Moving Castle,None,"Howl Pendragon, Sophie, Markl, Calcifer (Howl ...",...,0,0,0,0,0,0,0,0,0,0
1,The Life Plan,Anonfandom101,Life. ( A brief albeit important prolouge )\n\...,http://archiveofourown.org/works/19198501,Teen And Up Audiences,No Archive Warnings Apply,F/F,Hitchhiker's Guide to the Galaxy - Douglas Ada...,None,None,...,0,0,0,0,0,0,0,0,0,0
2,One Recurse,orphan_account,No one of them could be Cindi Mayweather. It's...,http://archiveofourown.org/works/333825,Mature,No Archive Warnings Apply,F/F,"Metropolis: The Chase Suite - Janelle Monáe, T...",None,"Alpha 9000s, Cindi Mayweather",...,0,0,0,0,0,0,0,0,0,0
3,"ask, and ye shall receive",Ariaste,"He feels rather daring about it, especially wh...",http://archiveofourown.org/works/19964368,General Audiences,No Archive Warnings Apply,M/M,"Good Omens (TV), Good Omens - Neil Gaiman & Te...",Aziraphale/Crowley (Good Omens),None,...,0,0,0,0,0,0,0,0,0,0
4,In Gardens,Bitterblue,"Neither of them is sure how to start, but Bit...",http://archiveofourown.org/works/432750,General Audiences,No Archive Warnings Apply,Gen,"Bitterblue - Kristin Cashore, Seven Kingdoms T...",None,"Hava (Bitterblue), Bitterblue (Graceling)",...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1011,Exchange,swiddershins,"He’d inquired about the coat’s price, carefull...",http://archiveofourown.org/works/14392434,General Audiences,No Archive Warnings Apply,Gen,"Neverwhere - Neil Gaiman, Neverwhere - All Med...",None,Marquis de Carabas (Neverwhere),...,0,0,0,0,0,0,0,0,0,0
1012,Peeta Mellark: In it to Survive,SilverTonguedWriter,The games were tomorrow. I had no fall back pl...,http://archiveofourown.org/works/1000334,Mature,Choose Not To Use Archive Warnings,M/M,Hunger Games Trilogy - Suzanne Collins,Cato/Peeta Mellark,"Cato (Hunger Games), Peeta Mellark",...,0,0,0,0,0,0,0,0,0,0
1013,Keeping Promises,fourredfruits,Everything happened so fast it seemed like the...,http://archiveofourown.org/works/18372194,General Audiences,Choose Not To Use Archive Warnings,F/M,"The Umbrella Academy (TV), The Umbrella Academ...","Number Five | The Boy & Vanya Hargreeves, Numb...","Number Five | The Boy (Umbrella Academy), Vany...",...,0,0,0,1,0,0,0,0,0,0
1014,Interlude: It grows back,kaleigh,,http://archiveofourown.org/works/21219572,Not Rated,No Archive Warnings Apply,F/F,"Addams Family - All Media Types, Addams Family...",Wednesday Addams/Original Female Character(s),"Wednesday Addams, Pubert Addams, Lurch (Addams...",...,0,0,0,0,0,0,0,0,0,0


In [4]:
def tokenize(text):
    # normalize case and remove punctuation
    text = re.sub(r"[^a-zA-Z0-9]", " ", text.lower())
    
    # tokenize text
    tokens = word_tokenize(text)
    
    # lemmatize
    tokens = [lemmatizer.lemmatize(word) for word in tokens]

    return tokens

In [17]:
ycols = ['tags_abductionTags', 'tags_ableismTags', 'tags_abuseGeneralTags', 'tags_academiaTags',
          'tags_addamsFamilyTags', 'tags_adventureTags', 'tags_ageTags', 'tags_alcoholAbuseTags',
        'tags_americanGodsTags', 'tags_americanHippoTags', 'tags_angstTags', 'tags_animalAbuseTags',
        'tags_apocalypseTags', 'tags_badRelationshipsTags', 'tags_bookThiefTags', 'tags_brotherWhereArtThouTags',
        'tags_characterOfColorTags', 'tags_childAbuseTags', 'tags_childRearingTags',
        'tags_christianity_jonahWhaleTags', 'tags_comedyTags', 'tags_comfortTags', 'tags_comingOfAgeTags',
        'tags_consensualViolenceTags', 'tags_cultureTags', 'tags_danceFunTags', 'tags_darkThemesTags',
        'tags_deathTags', 'tags_diannaWynneJonesTags', 'tags_disabilityTags', 'tags_domesticTags', 'tags_downerTags',
        'tags_dramaTags', 'tags_dubConTags', 'tags_dysphoriaTags', 'tags_dystopiaTags', 'tags_ellaEnchantedTags',
        'tags_enslavementTags', 'tags_eroticaSmutTags', 'tags_escapeRelatedTags', 'tags_explorationTags',
        'tags_failtoSaveWorldTags', 'tags_familyTags', 'tags_fascismTags', 'tags_fatherTags', 'tags_fatphobiaTags',
        'tags_feelGoodTags', 'tags_feministTags', 'tags_fightingTags', 'tags_funCrimeTags', 'tags_furryFandomTags',
        'tags_generalProblematicContentTags', 'tags_ghostbustersUniverseTags', 'tags_giverSeriesTags',
        'tags_godsJadeShadowTags', 'tags_goodOmensTags', 'tags_goreTags', 'tags_gracelingRealmTags',
        'tags_greatTerribleBeautyTags', 'tags_greatestShowmanMovieTags', 'tags_greysAnatomyTVTags', 
        'tags_harryPotterTags', 'tags_hhGttGTags', 'tags_hobbiesTags', 'tags_holidayTags', 'tags_horrorTags',
        'tags_howlsMovingCastleTags', 'tags_htbawTags', 'tags_hungerGamesTags', 'tags_hurtComfortTags',
        'tags_illnessTags', 'tags_inCryptidTags', 'tags_incestTags', 'tags_infidelityTags', 'tags_injuryRelatedTags',
        'tags_jrrTolkienTags', 'tags_lawEnforcementTags', 'tags_leverageTVTags', 'tags_lgbtqiaTags', 
        'tags_locationTags', 'tags_magicalRealismTags', 'tags_marriageTags', 'tags_marvelUniverseTags', 'tags_marvel_avengersTags',
        'tags_marvel_blackPantherTags', 'tags_marvel_blackWidowTags', 'tags_marvel_capAmericaTags', 'tags_marvel_guardiansGalaxyTags',
        'tags_marvel_hydraTags', 'tags_marvel_ironManTags', 'tags_marvel_misfitsTags', 'tags_marvel_msMarvelTags',
        'tags_marvel_thorTags', 'tags_marvel_xMenTags', 'tags_medicalEmergencyTags', 'tags_mentalCapacityTags',
        'tags_mentalIllnessesTags', 'tags_mentalViolenceTags', 'tags_mistakesAfootTags', 'tags_mistakesRelatedTags',
        'tags_motherTags', 'tags_mundaneUniverseTags', 'tags_musicTags', 'tags_mythosTags', 'tags_mythos_egyptMythTags',
        'tags_mythos_euroMythTags', 'tags_mythos_greekMythTags', 'tags_mythos_mayanMythTags', 'tags_mythos_norseMythTags',
        'tags_mythos_phillippineMythTags', 'tags_needContextTags', 'tags_neverwhereTags', 'tags_nightCircusBooksTags',
        'tags_notIdealTags', 'tags_pacificRimTags', 'tags_panTags', 'tags_parentTags', 'tags_perksWallflowerTags',
        'tags_peterPanTags', 'tags_physicalIllnessTags', 'tags_platonicRomanticTags', 'tags_platonicTags', 'tags_politicalTags',
        'tags_pregnancyRelatedTags', 'tags_queerphobiaTags', 'tags_racismTags', 'tags_radicalTags',
        'tags_relationshipsTags', 'tags_religionTags', 'tags_revengeTags', 'tags_revolutionRelatedTags', 'tags_romanceTags', 'tags_sameSexTags', 'tags_sassTags', 'tags_savingTheWorldTags', 'tags_sciFiTags', 'tags_selfHarmTags', 'tags_sexTags', 'tags_sexismTags', 'tags_sexualViolenceTags', 'tags_sherlockHolmesTags', 'tags_siblingsTags', 'tags_sisterTags', 'tags_sixthWorldTags', 'tags_spiritedAwayTags', 'tags_starWarsTags', 'tags_studioGhibliTags', 'tags_substanceAbuseTags', 'tags_suicideTags', 'tags_supernaturalTVTags', 'tags_supernaturalTags', 'tags_swearTags', 'tags_textLengthTags', 'tags_therapyWellnessTags', 'tags_timePeriodTags', 'tags_tired', 'tags_tortureTags', 'tags_transTags', 'tags_transphobiaTags', 'tags_traumaGeneralTags', 'tags_travelTags', 'tags_tropesTags', 'tags_umbrellaAcademyTags', 'tags_unmanagedMentalHealthTags', 'tags_unplesantEmotionTags', 'tags_vagrantQueenTags', 'tags_violenceGeneralTags', 'tags_violentDeathTags', 'tags_warTags', 'tags_wayfarersSeriesTags', 'tags_weaponGeneralTags', 'tags_werewolfTags', 'tags_womenCharactersTags', 'tags_wrinkleInTimeTags', 'tags_writingFormatTags', 'tags_xenophobiaTags',
        'fandomGenres_action', 'fandomGenres_apocalypse', 'fandomGenres_biography', 'fandomGenres_comedy', 'fandomGenres_comicsSource', 'fandomGenres_comingOfAge', 'fandomGenres_detective', 'fandomGenres_drama', 'fandomGenres_dystopia', 'fandomGenres_dystopian', 'fandomGenres_fantasy', 'fandomGenres_genreUnknown', 'fandomGenres_historicalFiction', 'fandomGenres_horror', 'fandomGenres_magicalRealism', 'fandomGenres_middleGrade', 'fandomGenres_multiGenres', 'fandomGenres_multigenerationalFamilyDrama', 'fandomGenres_mystery', 'fandomGenres_mythos', 'fandomGenres_paranormal', 'fandomGenres_portalFantasy', 'fandomGenres_realPersonFiction', 'fandomGenres_science', 'fandomGenres_scienceFantasy', 'fandomGenres_scienceFiction', 'fandomGenres_speculativeFiction', 'fandomGenres_superhero', 'fandomGenres_superheroes', 'fandomGenres_thriller', 'fandomGenres_urbanFantasy', 'fandomGenres_vampires', 'fandomGenres_youngAdult', 'fandomGenres_zombieGen',
        'consolidatedFandom_addamsFamilyTags', 'consolidatedFandom_americanGodsTags', 'consolidatedFandom_americanHippoTags', 
        'consolidatedFandom_bandomTags', 'consolidatedFandom_bintiTags', 'consolidatedFandom_blackDogGaimanTags', 
        'consolidatedFandom_boneStreetRumbaTags', 'consolidatedFandom_bookThiefTags', 'consolidatedFandom_cabinPressureTags', 
        'consolidatedFandom_catPicturesTags', 'consolidatedFandom_charlieChocFactoryTags', 'consolidatedFandom_christianityTags', 
        'consolidatedFandom_chroniclesNarniaTags', 'consolidatedFandom_cindiMayweatherTags', 'consolidatedFandom_deathStrandingTags', 
        'consolidatedFandom_diannaWynneJonesTags', 'consolidatedFandom_doctorWhoTags', 'consolidatedFandom_enchantedForestTags', 
        'consolidatedFandom_furryFandomTags', 'consolidatedFandom_gabrielMovieTags', 'consolidatedFandom_gaimanWorksTags', 
        'consolidatedFandom_gardenHeightsTags', 'consolidatedFandom_gargoylesCartoonTags', 'consolidatedFandom_ghostbustersUniverseTags', 
        'consolidatedFandom_ginenUniverseTags', 'consolidatedFandom_giverSeriesTags', 'consolidatedFandom_godsJadeShadowTags', 
        'consolidatedFandom_goodOmensTags', 'consolidatedFandom_gracelingRealmTags', 'consolidatedFandom_graveyardBookTags', 
        'consolidatedFandom_greatTerribleBeautyTags', 'consolidatedFandom_hamiltonTags', 'consolidatedFandom_harryPotterTags', 
        'consolidatedFandom_hauntingHillHouseTags', 'consolidatedFandom_hhGttGTags', 'consolidatedFandom_homegoingTags', 
        'consolidatedFandom_howlsMovingCastleTags', 'consolidatedFandom_htbawTags', 'consolidatedFandom_hungerGamesTags', 
        'consolidatedFandom_inCryptidTags', 'consolidatedFandom_joyLuckClubTags', 'consolidatedFandom_leverageTVTags', 
        'consolidatedFandom_magicSchoolBusTags', 'consolidatedFandom_marvelUniverseTags', 'consolidatedFandom_marvel_blackPantherTags', 
        'consolidatedFandom_marvel_blackWidowTags', 'consolidatedFandom_marvel_capAmericaTags', 'consolidatedFandom_marvel_capMarvelTags', 
        'consolidatedFandom_marvel_cloakDaggerTags', 'consolidatedFandom_marvel_daredevilTags', 'consolidatedFandom_marvel_drStrangeTags', 
        'consolidatedFandom_marvel_fantasticFourTags', 'consolidatedFandom_marvel_guardiansGalaxyTags', 'consolidatedFandom_marvel_ironManTags', 
        'consolidatedFandom_marvel_lukeCageTags', 'consolidatedFandom_marvel_msMarvelTags', 'consolidatedFandom_marvel_spiderManTags', 
        'consolidatedFandom_marvel_thorTags', 'consolidatedFandom_marvel_xMenTags', 
        'consolidatedFandom_matildaTags', 'consolidatedFandom_monarchGlenTags', 'consolidatedFandom_mythosTags', 
        'consolidatedFandom_neverwhereTags', 'consolidatedFandom_newsfleshTags', 'consolidatedFandom_nightCircusBooksTags', 
        'consolidatedFandom_nkJemisinTags', 'consolidatedFandom_originalWorkTags', 'consolidatedFandom_pacificRimTags', 
        'consolidatedFandom_perksWallflowerTags', 'consolidatedFandom_potcTags', 'consolidatedFandom_sandmanComicsTags', 
        'consolidatedFandom_sherlockHolmesTags', 'consolidatedFandom_sixthWorldTags', 
        'consolidatedFandom_spiritedAwayTags', 'consolidatedFandom_starTrekTags', 'consolidatedFandom_starWarsTags', 
        'consolidatedFandom_superheroGroupsTags', 'consolidatedFandom_supernaturalTVTags', 'consolidatedFandom_theBellesTags',
        'consolidatedFandom_thePowerTags', 'consolidatedFandom_trumanShowTags', 'consolidatedFandom_umbrellaAcademyTags',
        'consolidatedFandom_underworldTags', 'consolidatedFandom_vagrantQueenTags', 'consolidatedFandom_wayfarersSeriesTags', 
        'consolidatedFandom_whiteCollarTags', 'consolidatedFandom_worstWitchTags', 'consolidatedFandom_wrinkleInTimeTags']

In [5]:
X = df.story_text
y = df[ycols]

In [6]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [8]:
pipeline = Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer()),
            ('clf', MultiOutputClassifier(AdaBoostClassifier()))
            ])

In [9]:
# train classifier
pipeline.fit(X_train, y_train)

Pipeline(steps=[('vect',
                 CountVectorizer(tokenizer=<function tokenize at 0x7f94a971daf0>)),
                ('tfidf', TfidfTransformer()),
                ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))])

In [10]:
pipeline.get_params()

{'memory': None,
 'steps': [('vect',
   CountVectorizer(tokenizer=<function tokenize at 0x7f94a971daf0>)),
  ('tfidf', TfidfTransformer()),
  ('clf', MultiOutputClassifier(estimator=AdaBoostClassifier()))],
 'verbose': False,
 'vect': CountVectorizer(tokenizer=<function tokenize at 0x7f94a971daf0>),
 'tfidf': TfidfTransformer(),
 'clf': MultiOutputClassifier(estimator=AdaBoostClassifier()),
 'vect__analyzer': 'word',
 'vect__binary': False,
 'vect__decode_error': 'strict',
 'vect__dtype': numpy.int64,
 'vect__encoding': 'utf-8',
 'vect__input': 'content',
 'vect__lowercase': True,
 'vect__max_df': 1.0,
 'vect__max_features': None,
 'vect__min_df': 1,
 'vect__ngram_range': (1, 1),
 'vect__preprocessor': None,
 'vect__stop_words': None,
 'vect__strip_accents': None,
 'vect__token_pattern': '(?u)\\b\\w\\w+\\b',
 'vect__tokenizer': <function __main__.tokenize(text)>,
 'vect__vocabulary': None,
 'tfidf__norm': 'l2',
 'tfidf__smooth_idf': True,
 'tfidf__sublinear_tf': False,
 'tfidf__use_idf

In [11]:
y_pred = pipeline.predict(X_test)
ycols = list(y.columns)
print(classification_report(y_test, y_pred, target_names=ycols))
dystopian, fantasy, portal fantasy, magical realism, science fiction, superheroes, urbanFantasy, young adult

                                               precision    recall  f1-score   support

                           tags_abductionTags       0.00      0.00      0.00         0
                             tags_ableismTags       0.00      0.00      0.00         0
                        tags_abuseGeneralTags       0.00      0.00      0.00         1
                            tags_academiaTags       0.00      0.00      0.00         4
                        tags_addamsFamilyTags       0.00      0.00      0.00         2
                           tags_adventureTags       1.00      1.00      1.00         1
                                 tags_ageTags       0.00      0.00      0.00         2
                        tags_alcoholAbuseTags       0.50      0.11      0.18         9
                        tags_americanGodsTags       0.50      0.22      0.31         9
                       tags_americanHippoTags       0.00      0.00      0.00         0
                               tags_angstT

/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in samples with no predicted labels. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [14]:
#tested parameters
"""parameters = {'vect__analyzer': ['word', 'char', 'char_wb'],
              'vect__stop_words': ['english', None],
              'vect__ngram_range': [(1, 1), (1, 2)],
              'clf__estimator__algorithm': ['SAMME', 'SAMME.R'],
              'clf__n_jobs': [1, 3]
             }"""

#amended parameters to make gridsearch run a little faster based on the best estimator
parameters = {'vect__stop_words': ['english', None],
              'clf__estimator__algorithm': ['SAMME', 'SAMME.R'],
              'clf__n_jobs': [1, 3]
             }

grid_search_pipeline = GridSearchCV(pipeline, param_grid=parameters, verbose=3)

In [15]:
#look for improvements via gridsearch
grid_search_pipeline.fit(X_train, y_train)

Fitting 5 folds for each of 48 candidates, totalling 240 fits
[CV] clf__estimator__algorithm=SAMME, clf__n_jobs=1, vect__analyzer=word, vect__ngram_range=(1, 1), vect__stop_words=english 


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[CV]  clf__estimator__algorithm=SAMME, clf__n_jobs=1, vect__analyzer=word, vect__ngram_range=(1, 1), vect__stop_words=english, score=0.052, total= 2.6min
[CV] clf__estimator__algorithm=SAMME, clf__n_jobs=1, vect__analyzer=word, vect__ngram_range=(1, 1), vect__stop_words=english 


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:  2.6min remaining:    0.0s
/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


[CV]  clf__estimator__algorithm=SAMME, clf__n_jobs=1, vect__analyzer=word, vect__ngram_range=(1, 1), vect__stop_words=english, score=0.052, total= 2.5min
[CV] clf__estimator__algorithm=SAMME, clf__n_jobs=1, vect__analyzer=word, vect__ngram_range=(1, 1), vect__stop_words=english 


...

/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 431, in _process_worker
    r = call_item()
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 285, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/_parallel_backends.py", line 595, in __call__
    return self.func(*args, **kwargs)
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 252, in __call__
    return [func(*args, **kwargs)
  File "/Users/jbryant/opt/anaconda3/li

[CV]  clf__estimator__algorithm=SAMME.R, clf__n_jobs=3, vect__analyzer=char_wb, vect__ngram_range=(1, 2), vect__stop_words=None, score=nan, total=   4.2s
[CV] clf__estimator__algorithm=SAMME.R, clf__n_jobs=3, vect__analyzer=char_wb, vect__ngram_range=(1, 2), vect__stop_words=None 


/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/model_selection/_validation.py:548: FitFailedWarning: Estimator fit failed. The score on this train-test partition for these parameters will be set to nan. Details: 
joblib.externals.loky.process_executor._RemoteTraceback: 
"""
Traceback (most recent call last):
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 431, in _process_worker
    r = call_item()
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/externals/loky/process_executor.py", line 285, in __call__
    return self.fn(*self.args, **self.kwargs)
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/_parallel_backends.py", line 595, in __call__
    return self.func(*args, **kwargs)
  File "/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/joblib/parallel.py", line 252, in __call__
    return [func(*args, **kwargs)
  File "/Users/jbryant/opt/anaconda3/li

[CV]  clf__estimator__algorithm=SAMME.R, clf__n_jobs=3, vect__analyzer=char_wb, vect__ngram_range=(1, 2), vect__stop_words=None, score=nan, total=   4.2s


GridSearchCV(estimator=Pipeline(steps=[('vect',
                                        CountVectorizer(tokenizer=<function tokenize at 0x7f94a971daf0>)),
                                       ('tfidf', TfidfTransformer()),
                                       ('clf',
                                        MultiOutputClassifier(estimator=AdaBoostClassifier()))]),
             param_grid={'clf__estimator__algorithm': ['SAMME', 'SAMME.R'],
                         'clf__n_jobs': [1, 3],
                         'vect__analyzer': ['word', 'char', 'char_wb'],
                         'vect__ngram_range': [(1, 1), (1, 2)],
                         'vect__stop_words': ['english', None]},
             verbose=3)

In [18]:
grid_search_pipeline.best_estimator_

Pipeline(steps=[('vect',
                 CountVectorizer(stop_words='english',
                                 tokenizer=<function tokenize at 0x7f94a971daf0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME'),
                                       n_jobs=1))])

In [19]:
#update pipeline based on gridsearch results
pipeline = Pipeline([('vect', CountVectorizer(stop_words='english', tokenizer=tokenize)),
                     ('tfidf', TfidfTransformer()),
                     ('clf', MultiOutputClassifier(AdaBoostClassifier(algorithm='SAMME'),
                                                   n_jobs=1))])

In [20]:
# train classifier
pipeline.fit(X_train, y_train)

/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/feature_extraction/text.py:383: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['ha', 'le', 'u', 'wa'] not in stop_words.
  warnings.warn('Your stop_words may be inconsistent with '


Pipeline(steps=[('vect',
                 CountVectorizer(stop_words='english',
                                 tokenizer=<function tokenize at 0x7f94a971daf0>)),
                ('tfidf', TfidfTransformer()),
                ('clf',
                 MultiOutputClassifier(estimator=AdaBoostClassifier(algorithm='SAMME'),
                                       n_jobs=1))])

In [21]:
#make predictions
y_pred = pipeline.predict(X_test)

In [22]:
#check scores for predictions
print(classification_report(y_test, y_pred, target_names=ycols))

                                               precision    recall  f1-score   support

                           tags_abductionTags       0.00      0.00      0.00         0
                             tags_ableismTags       0.00      0.00      0.00         0
                        tags_abuseGeneralTags       0.00      0.00      0.00         1
                            tags_academiaTags       0.00      0.00      0.00         4
                        tags_addamsFamilyTags       0.00      0.00      0.00         2
                           tags_adventureTags       1.00      1.00      1.00         1
                                 tags_ageTags       0.00      0.00      0.00         2
                        tags_alcoholAbuseTags       0.00      0.00      0.00         9
                        tags_americanGodsTags       0.67      0.22      0.33         9
                       tags_americanHippoTags       0.00      0.00      0.00         0
                               tags_angstT

/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/jbryant/opt/anaconda3/lib/python3.8/site-packages/sklearn/metrics/_classification.py:1221: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


In [28]:
#save model
with open('fictionTaggingEngine.pkl', 'wb') as file:
        pickle.dump(pipeline, file)